# Photosynthesis, multi-leaf tutorial

This tutorial is for analyzing photosynthetic efficiency for multiple leaves.

# Section 1: Importing Image and Libraries

In [ ]:
# Set the notebook display method
# If widget is not working, then change to inline
%matplotlib widget

# Import libraries
from plantcv import plantcv as pcv
from plantcv.parallel import WorkflowInputs

# Print out the version of PlantCV being used by the Jupyter kernel
pcv.__version__

PSII images (3 in a set; F0, Fmin, and Fmax) are captured directly following a saturating fluorescence pulse (red light; 630 nm). These three PSII images can be used to calculate Fv/Fm (efficiency of photosystem II) for each pixel of the plant. 

To run a PSII workflow over a single PSII image set (3 images) there are 4 required inputs:

Image 1: F0 (a.k.a Fdark/null) image.
Image 2: Fmin image.
Image 3: Fmax image.
Output directory: If debug mode is set to 'print' output images from each step are produced.

## Input/Output variables

The options class mimics the workflow command-line argument parser that is used for workflow parallelization. Using it while developing a workflow in Jupyter makes it easier to convert the workflow to a script later. Remember, always keep your raw images separate from your newly processed images!

In [ ]:
# Input/output options
args = WorkflowInputs(
    images=["./imgs/PSII_HDR_2023-12-11_B01_1.INF"],
    names="image1",
    result="single-plant-results.csv",
    outdir="./example-results",
    writeimg=True,
    debug="plot"
    )

In [ ]:
# Set debug to the global parameter 
pcv.params.debug = args.debug
# Change display settings
pcv.params.dpi = 100
# Increase text size and thickness to make labels clearer
# (size may need to be altered based on original image size)
pcv.params.text_size = 20
pcv.params.text_thickness = 20

## Read the input image

### Reading images into your environment using *pcv.readimage()*
Inputs:
   * filename = Image file to be read in
   * mode     = How the image will be read into the notebook; either 'native' (default), 'rgb', 'gray', 'csv', or 'envi'

In [ ]:
img, path, filename = pcv.readimage(filename=args.image1)

The CropReporter will output multiple data files, depending on the measurement protocols that are activated on the instrument.

### INF file
**HDR** = the configuration file that describes what measurements were on/off and other settings. This is the input file used by PlantCV

### DAT files
**PSD** = Dark-adapted chlorophyll fluorescence measurements following a saturating light pulse (red light; 630 nm). These images are used to calculate Fv/Fm (efficiency of photosystem II).

**PSL** = Light-adapted chlorophyll fluorescence measurements following a saturating light pulse (red light; 630 nm). These images are used to calculate Fq'/Fm' (operating efficiency of photosystem II).

PSD and PSL are used together to calculate non-photochemical quenching (NPQ).

**CLR** = Blue (475 nm), Green (550 nm), and Red (640 nm) color channels.

**CHL** = Steady-state chlorophyll fluorescence.

**SPC** = Spectral channels used to calculate vegetative indices. Green2 (540 nm), Far-red (710 nm), and Near-infrared (770 nm).

ps is an instance of the PSII_data class in PlantCV. The class stores each available dataset as attributes. The class stores two dataset attributes (datapath and filename) and stores each of the datasets with the following variable names: ojip_dark, ojip_light, chlorophyll, spectral. The ojip_dark, ojip_light, and chlorophyll datasets are stored as xarray DataArrays. The spectral dataset is stored as a PlantCV Spectral_data class instance.

The spectral dataset can contain blue (460nm), green (500nm), red (670nm), green2 (550nm), far-red (700nm), and near-infrared (800nm) frames.


In [ ]:
# Read fluorescence image data
# This will visualize all of the frames in all of the data you collected

# Inputs:
#   filename - Image file to be read in (should be an .INF file)
ps = pcv.photosynthesis.read_cropreporter(filename=args.image1)


## Create a mask

In the code below, we select the chlorophyll "CHL" frame as our grayscale image from which the mask will be created. It is possible to use any frame for creating a mask, but it is easier to use an image with decent contrast between the object of interest (the plant) and the background. 

In [ ]:
img = ps.chlorophyll.sel(frame_label = "Chl").data
pcv.plot_image(img)

In [ ]:
# Segment the chlorophyll image in order to separate the plant from the background
# For additional segmentation methods (such as binary), see the segmentation tutorial and documentation

# Inputs:
#   gray_img        - Grayscale image data
#   object_type     - 'light' (default) or 'dark'. If the object is lighter than the
#                       background then standard threshold is done. If the object is
#                       darker than the background then inverse thresholding is done.

bin_img = pcv.threshold.otsu(gray_img=img, object_type="light")


In [ ]:
# Fill small objects to remove noise and get a complete plant

# Inputs:
#   bin_img         - Binary image data
#   size            - Minimum object area size in pixels (integer), smaller objects get filled in.

filled_mask = pcv.fill(bin_img=bin_img, size=200)


Segmentation accuracy depends largely on the quality of the imaging data collection setup, but the importance of this accuracy also depends on the experimental questiton that a workflow aims to answer. In the case of photosynthesis/fluorescence datasets, it's recommended that segmentation of plants be more conservative. In other words, it's more detrimental for PSII results to have background signal captured within a plant mask than it is to exclude some true plant signal in analysis. Plant movement, reflections within imaging cabinets, and other factors can influence the signal around the edges of an object. To address this, we will preform a morphological erosion to remove the edges.  

In [ ]:
# Erode a small border of pixels from the mask 

# Inputs:
#   gray_img - Grayscale (usually binary) image data 
#   ksize - The size used to build a ksize x ksize 
#            matrix using np.ones. Must be greater than 1 to have an effect 
#   i - An integer for the number of iterations 
eroded_mask = pcv.erode(gray_img=filled_mask, ksize=3, i=1)


## Create a labeled mask

We will create a labeled mask with a unique label for each leaf. Because we want to keep track of the identity of each leaf we will use multiple regions of interest (ROIs) so that we can assign a label to each leaf by position.

In [ ]:
#     Create multiple circular ROIs on a single image
# Inputs
# img           = Input image data.
# coord         = Two-element tuple of the center of the top left object (x,y) or a list of tuples identifying
#                 the center of each roi [(x1,y1),(x2,y2)]
# radius        = A single radius for all ROIs.
# spacing       = Two-element tuple of the horizontal and vertical spacing between ROIs, (x,y). Ignored if `coord`
#                 is a list and `rows` and `cols` are None.
# nrows         = Number of rows in ROI layout. Should be missing or None if each center coordinate pair is listed.
# ncols         = Number of columns in ROI layout. Should be missing or None if each center coordinate pair is listed.
# Returns:
# roi_objects   = a dataclass with roi objects and hierarchies
rois = pcv.roi.multi(img=eroded_mask, coord=(600, 160), radius=30, spacing=(0, 115), nrows=7, ncols=1)


In [ ]:
# Create a labeled mask
# Inputs:
# mask            = mask image
# rois            = list of multiple ROIs (from roi.multi or roi.auto_grid)
# roi_type        = 'cutto', 'partial' (for partially inside, default),
#                 'largest' (keep only the largest contour), or 'auto'
#                 (use the mask alone withtout roi filtering)

# Returns:
# mask            = Labeled mask
# num_labels      = Number of labeled objects
labeled_mask, n_labels = pcv.create_labels(mask=filled_mask, rois=rois)

## Phenotypic measurements

Label each object according to the known identity. 

In [ ]:
# Sample label list
labels = ["Floor-Pot_1", "Floor-Pot_2", "Floor-Pot_3", "Floor-Pot_4", "Floor-Pot_5", "Floor-Pot_6","Floor-Pot_7"]

In [ ]:
# Analyze shape and size of each leaf
shape_img = pcv.analyze.size(img=img, labeled_mask=labeled_mask, n_labels=n_labels, label=labels)

### Visualize the chlorophyll fluorescence induction curves (optional)

In this experiment, the leaves were dark-adapted. An image is taken of the leaves in the dark (F-dark). The leaves were then exposed to a saturating red light pulse briefly. An image of chlorophyll fluorescence is taken immediately after the pulse to measure minimal fluorescence (F0). Successive images are taken at a fixed time interval (20 total frames from F0 to F19 in this example). Here we will chart the induction curves using visualize.chlorophyll_fluorescence to see if the maximum fluorescence frames are set at a reasonable place or whether we want to adjust them in a later step.

In [ ]:
# Dark-adapted fluorescence induction curve
# This curve is only for the object in your kept mask from above

# Inputs:
# ps_da            = photosynthesis xarray DataArray
# labeled_mask     = Labeled mask of objects (32-bit).
# n_labels         = Total number expected individual objects (default = 1).
# label            = optional label parameter, modifies the prefix of the group plotting label

# Returns:
# chart            = Plot of the chlorophyll fluorescence induction curve for each object

dark_fig = pcv.visualize.chlorophyll_fluorescence(ps_da=ps.ojip_dark, labeled_mask=labeled_mask, n_labels=n_labels)

After the Fv/Fm protocol, the leaves are light-adapted. An image is taken of the leaves in the light (F-light). The leaves are then exposed to an actinic light, and an image of chlorophyll fluorescence is taken to measure steady-state fluorescence (F'). The leaves are then exposed to a saturating red light pulse briefly. Successive images are taken at a fixed time interval (20 total frames from F0 to F19 in this example). Here we will use the visualize.chlorophyll_fluorescence function to chart the fluorescence induction curve.

In [ ]:
# Light-adapted fluorescence induction curve

# Inputs:
# ps_da            = photosynthesis xarray DataArray
# labeled_mask     = Labeled mask of objects (32-bit).
# n_labels         = Total number expected individual objects (default = 1).
# label            = optional label parameter, modifies the prefix of the group plotting label

# Returns:
# chart            = Plot of the chlorophyll fluorescence induction curve for each object

light_fig = pcv.visualize.chlorophyll_fluorescence(ps_da=ps.ojip_light, labeled_mask=labeled_mask, n_labels=n_labels)

# Estimate the efficiency of PSII

Using the dark-adapted and light-adapted datasets, calculate the metric Fv/Fm to estimate the maximum efficiency of PSII and the metric Fq'/Fm' to estimate the operating efficiency of PSII. In both cases the function analyze.yii is used. Setting auto_fm=True will use photosynthesis.reassign_frame_labels to automatically find the frame with the maximum fluorescence for each masked region.

In [ ]:
# Analyze Fv/Fm

# Inputs:
# ps_da               = Photosynthesis xarray DataArray (either ojip_dark or ojip_light)
# labeled_mask        = Labeled mask of objects (32-bit).
# n_labels            = Total number expected individual objects (default = 1).
# auto_fm             = Automatically calculate the frame with maximum fluorescence per label, otherwise
#                       use a fixed frame for all labels (default = False).
# measurement_labels  = labels for each measurement, modifies the variable name of observations recorded
# label               = optional label parameter, modifies the variable name of observations recorded

# Returns:
# yii_global          = DataArray of efficiency estimate values
# yii_chart           = Histograms of efficiency estimate

fvfm, fvfm_hist = pcv.analyze.yii(ps_da=ps.ojip_dark, labeled_mask=labeled_mask, n_labels=n_labels, auto_fm=True,
                                  measurement_labels=["Fv/Fm"], label=labels)


In [ ]:
# Analyze Fq'/Fm'

# Inputs:
# ps_da               = Photosynthesis xarray DataArray (either ojip_dark or ojip_light)
# labeled_mask        = Labeled mask of objects (32-bit).
# n_labels            = Total number expected individual objects (default = 1).
# auto_fm             = Automatically calculate the frame with maximum fluorescence per label, otherwise
#                       use a fixed frame for all labels (default = False).
# measurement_labels  = labels for each measurement, modifies the variable name of observations recorded
# label               = optional label parameter, modifies the variable name of observations recorded

# Returns:
# yii_global          = DataArray of efficiency estimate values
# yii_chart           = Histograms of efficiency estimate

fqfm, fqfm_hist = pcv.analyze.yii(ps_da=ps.ojip_light, labeled_mask=labeled_mask, n_labels=n_labels, auto_fm=True,
                                  measurement_labels=["Fq'/Fm'"], label=labels)

# Estimate non-photochemical quenching 

Nonphotochemical quanching (NPQ) can be estimated using the analyze.npq function.

In [ ]:
# Analyze NPQ

# Inputs:
# ps_da_light        = Photosynthesis xarray DataArray that contains frame_label `Fmp` (ojip_light)
# ps_da_dark         = Photosynthesis xarray DataArray that contains frame_label `Fm` (ojip_dark)
# labeled_mask       = Labeled mask of objects (32-bit).
# n_labels           = Total number expected individual objects (default = 1).
# auto_fm            = Automatically calculate the frame with maximum fluorescence per label, otherwise
#                      use a fixed frame for all labels (default = False).
# min_bin            = minimum bin value ("auto" or user input minimum value - must be an integer)
# max_bin            = maximum bin value ("auto" or user input maximum value - must be an integer)
# measurement_labels = labels for each measurement in ps_da_light, modifies the variable name of observations recorded
# label              = optional label parameter, modifies the entity name of observations recorded

# Returns:
# npq_global         = DataArray of NPQ values
# npq_chart          = Histograms of NPQ estimates

npq, npq_hist = pcv.analyze.npq(ps_da_light=ps.ojip_light, ps_da_dark=ps.ojip_dark, labeled_mask=labeled_mask, n_labels=n_labels,
                                auto_fm=True, measurement_labels=["NPQ"], label=labels, max_bin=10)

Optionally, the Fv/Fm, Fq'/Fm', and NPQ images can be visualized using the pseudocolor function to assess the distribution of values across the leaves.

In [ ]:
# Pseudocolor the PSII metric images

# Inputs:
# gray_img    - grayscale image data
# obj         - (optional) ROI or plant contour object. If provided, the pseudocolored image gets cropped
#               down to the region of interest. default = None
# mask        - (optional) binary mask
# cmap        - (optional) colormap. default is the matplotlib default, viridis
# background  - (optional) background color/type, options are "image" (gray_img), "white", or "black"
#               (requires a mask). default = 'image'
# min_value   - (optional) minimum value for range of interest. default = 0
# max_value   - (optional) maximum value for range of interest. default = 255
# axes        - (optional) if False then x- and y-axis won't be displayed, nor will the title. default = True
# colorbar    - (optional) if False then colorbar won't be displayed. default = True
# obj_padding - (optional) if "auto" (default) and an obj is supplied, then the image is cropped to an extent 20%
#               larger in each dimension than the object. An single integer is also accepted to define the padding
#               in pixels
# title       - (optional) custom title for the plot gets drawn if title is not None. default = None
# bad_mask    - (optional) binary mask of pixels with "bad" values, e.g. nan or inf or any other values considered
#               to be not informative and to be excluded from analysis. default = None
# bad_color   - (optional) desired color to show "bad" pixels. default = "red"
fvfm_cmap = pcv.visualize.pseudocolor(gray_img=fvfm, mask=filled_mask, cmap="viridis", 
                                        min_value=0, max_value=1, title="Fv/Fm")
fqfm_cmap = pcv.visualize.pseudocolor(gray_img=fqfm, mask=filled_mask, cmap="viridis", 
                                        min_value=0, max_value=1, title="Fq'/Fm'")
npq_cmap = pcv.visualize.pseudocolor(gray_img=npq, mask=filled_mask, cmap="viridis", 
                                        min_value=0, max_value=3, title="NPQ")

# Analyze spectral indices

# Anthocyanin Reflectance Index
Calculate ARI using the ari function, plot a colormap, and analyze the leaf values.

In [ ]:
# Inputs:
# hsi         = hyperspectral image (PlantCV Spectral_data instance)
# distance    = how lenient to be if the required wavelengths are not available
# 
# Returns:
# index_array = Index data as a Spectral_data instance
ari = pcv.spectral_index.ari(hsi=ps.spectral)

ari_ps = pcv.visualize.pseudocolor(gray_img=ari.array_data, min_value=-10, max_value=5, 
                                    cmap="Purples", mask=filled_mask, background="black", 
                                    title="Anthocyanin Reflectance Index")
# Inputs:
# index_img    = Index image data (PlantCV Spectral_data object)
# labeled_mask = Labeled mask of objects (32-bit).
# n_labels     = Total number expected individual objects (default = 1).
# bins         = Number of histogram bins (default = 100)
# min_bin      = Minimum bin value (default = 0). "auto" will use the minimum value of the index image.
# max_bin      = Maximum bin value (default = 1). "auto" will use the maximum value of the index image.
# label        = optional label parameter, modifies the variable name of observations recorded (default = "default").

# Returns:
# index_hist = Spectral index histogram plot
ari_hist = pcv.analyze.spectral_index(index_img=ari, labeled_mask=labeled_mask, n_labels=n_labels, min_bin=-10, max_bin=5, label=labels)


# Chlorophyll Index Red Edge

Calculate CI using the ci_rededge function, plot a colormap, and analyze the leaf values.

In [ ]:
# Inputs:
# hsi         = hyperspectral image (PlantCV Spectral_data instance)
# distance    = how lenient to be if the required wavelengths are not available
# 
# Returns:
# index_array = Index data as a Spectral_data instance
ci = pcv.spectral_index.ci_rededge(hsi=ps.spectral, distance=30)

ci_ps = pcv.visualize.pseudocolor(gray_img=ci.array_data, min_value=0, max_value=8, 
                                    cmap="Greens", mask=filled_mask, background="black", 
                                    title="Chlorophyll Index Red Edge")
# Inputs:
# index_img    = Index image data (PlantCV Spectral_data object)
# labeled_mask = Labeled mask of objects (32-bit).
# n_labels     = Total number expected individual objects (default = 1).
# bins         = Number of histogram bins (default = 100)
# min_bin      = Minimum bin value (default = 0). "auto" will use the minimum value of the index image.
# max_bin      = Maximum bin value (default = 1). "auto" will use the maximum value of the index image.
# label        = optional label parameter, modifies the variable name of observations recorded (default = "default").

# Returns:
# index_hist = Spectral index histogram plot
ci_hist = pcv.analyze.spectral_index(index_img=ci, labeled_mask=labeled_mask, n_labels=n_labels, min_bin=0, max_bin=12, label=labels)

# Normalized Difference Vegetation Index
Calculate NDVI using the ndvi function, plot a colormap, and analyze the leaf values.

In [ ]:
# Inputs:
# hsi         = hyperspectral image (PlantCV Spectral_data instance)
# distance    = how lenient to be if the required wavelengths are not available
# 
# Returns:
# index_array = Index data as a Spectral_data instance
ndvi = pcv.spectral_index.ndvi(hsi=ps.spectral, distance=30)

ndvi_ps = pcv.visualize.pseudocolor(gray_img=ndvi.array_data, min_value=0, max_value=1, 
                                    cmap="jet", mask=filled_mask, background="black", 
                                    title="Normalized Difference Vegetation Index")
# Inputs:
# index_img    = Index image data (PlantCV Spectral_data object)
# labeled_mask = Labeled mask of objects (32-bit).
# n_labels     = Total number expected individual objects (default = 1).
# bins         = Number of histogram bins (default = 100)
# min_bin      = Minimum bin value (default = 0). "auto" will use the minimum value of the index image.
# max_bin      = Maximum bin value (default = 1). "auto" will use the maximum value of the index image.
# label        = optional label parameter, modifies the variable name of observations recorded (default = "default").

# Returns:
# index_hist = Spectral index histogram plot
ndvi_hist = pcv.analyze.spectral_index(index_img=ndvi, labeled_mask=labeled_mask, n_labels=n_labels, min_bin=0, max_bin=1, label=labels)


# Save results and finish the workflow¶

In [ ]:
# The save results function will take the measurements stored when running any PlantCV analysis functions,
# format, and print an output text file for data analysis. The Outputs class stores data whenever any of
# the following functions are ran: analyze_bound_horizontal, analyze_bound_vertical, analyze_color,
# analyze_nir_intensity, analyze_object, fluor_fvfm, report_size_marker_area, watershed. If no functions
# have been run, it will print an empty text file 

# This saves results for one image, and each image is saved individually if you run another image
# (it will overwrite the last one)
pcv.outputs.save_results(filename=args.result, outformat="csv")

if args.writeimg:
    pcv.print_image(img=dark_fig, filename=os.path.join(args.outdir, f"{ps.filename[:-4]}_fvfm_induction.png"))
    pcv.print_image(img=light_fig, filename=os.path.join(args.outdir, f"{ps.filename[:-4]}_fqfm_induction.png"))
    pcv.print_image(img=fvfm_hist, filename=os.path.join(args.outdir, f"{ps.filename[:-4]}_fvfm_histogram.png"))
    pcv.print_image(img=fqfm_hist, filename=os.path.join(args.outdir, f"{ps.filename[:-4]}_fqfm_histogram.png"))
    pcv.print_image(img=npq_hist, filename=os.path.join(args.outdir, f"{ps.filename[:-4]}_npq_histogram.png"))
    pcv.print_image(img=fvfm_cmap, filename=os.path.join(args.outdir, f"{ps.filename[:-4]}_fvfm_cmap.png"))
    pcv.print_image(img=fqfm_cmap, filename=os.path.join(args.outdir, f"{ps.filename[:-4]}_fqfm_cmap.png"))
    pcv.print_image(img=npq_cmap, filename=os.path.join(args.outdir, f"{ps.filename[:-4]}_npq_cmap.png"))
    pcv.print_image(img=ari_ps, filename=os.path.join(args.outdir, f"{ps.filename[:-4]}_ari_cmap.png"))
    pcv.print_image(img=ari_hist, filename=os.path.join(args.outdir, f"{ps.filename[:-4]}_ari_hist.png"))
    pcv.print_image(img=ci_ps, filename=os.path.join(args.outdir, f"{ps.filename[:-4]}_ci_cmap.png"))
    pcv.print_image(img=ci_hist, filename=os.path.join(args.outdir, f"{ps.filename[:-4]}_ci_hist.png"))
    pcv.print_image(img=ndvi_ps, filename=os.path.join(args.outdir, f"{ps.filename[:-4]}_ndvi_cmap.png"))
    pcv.print_image(img=ndvi_hist, filename=os.path.join(args.outdir, f"{ps.filename[:-4]}_ndvi_hist.png"))


In [ ]:
# Clear the measurements stored globally into the Ouptuts class

#If you have additional multi-object images, go back to the start of the workflow and start again. Clear the outputs each time an image is analyzed. 

pcv.outputs.clear()

After you have finished analyzing all of your images, you can combine the resulting csv files. Be sure each object has unique labels. You can then take the combined results file for downstream analysis of the numerical data. 

In [ ]:
# define the path to the individual result .csv files
path = './example-results'
os.chdir(path)
import glob
import pandas as pd

#select which files should be combined
all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]

#combine the individual .csv files
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_csv.to_csv( "combined_results.csv", index=False, encoding='utf-8-sig')